# Código Python para buscar publicaçoes Youtube e Gerar os arquivos .csv no formato normalizado e com label 

In [ ]:
# Importando as bibliotecas necessárias
import os
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from googleapiclient.discovery import build


In [ ]:
# Buscar o metadados dos videos postados conforme parametros de busca e data e hora
def search_youtube_videos(api_key, query, published_after, published_before):
    youtube = build('youtube', 'v3', developerKey=api_key)
    videos = []
    next_page_token = None
    
    while True:
        try:
            response = youtube.search().list(
                q=query,
                part='snippet',
                type='video',
                regionCode='BR',
                publishedAfter=published_after,
                publishedBefore=published_before,
                maxResults=50,  # Máximo de resultados por página
                pageToken=next_page_token
            ).execute()
        except Exception as e:
            print(f"Erro ao buscar vídeos: {e}")
            raise
        
        for item in response.get('items', []):
            video_id = item['id']['videoId']
            video_url = f"https://www.youtube.com/watch?v={video_id}"
            
            # Obter detalhes adicionais do vídeo
            try:
                video_response = youtube.videos().list(
                    part='snippet,statistics,contentDetails',
                    id=video_id
                ).execute()
                
                video_info = video_response['items'][0]
                
                # Verificar se os comentários estão desabilitados
                if 'comments' not in video_info['snippet']['defaultLanguage']:
                    print(f"Comentários desabilitados para o vídeo: {video_id}")
                    continue
                
                video_data = {
                    'video_id': video_id,
                    'url': video_url,
                    'title': video_info['snippet']['title'],
                    'description': video_info['snippet']['description'],
                    'channel_title': video_info['snippet']['channelTitle'],
                    'published_at': video_info['snippet']['publishedAt'],
                    'view_count': video_info['statistics'].get('viewCount', 0),
                    'like_count': video_info['statistics'].get('likeCount', 0),
                    'dislike_count': video_info['statistics'].get('dislikeCount', 0),
                    'comment_count': video_info['statistics'].get('commentCount', 0),
                    'tags': video_info['snippet'].get('tags', []),
                    'duration': video_info['contentDetails']['duration'],
                    'sentiment': None  # Nenhuma informação disponível na API sobre sentimento
                }
                videos.append(video_data)
            except Exception as e:
                print(f"Erro ao obter detalhes do vídeo {video_id}: {e}")
                continue
        
        next_page_token = response.get('nextPageToken')
        
        if not next_page_token:
            break
    
    return videos

In [ ]:
# Organiza o metadado conforme nossa normalização

def organize_video_data(videos):
    organized_data = []
    for video in videos:
        # Convertendo o valor numérico de SENTIMENT para a string correspondente
        sentiment_mapping = {0: 'NEUTRAL', 1: 'POSITIVE', 2: 'NEGATIVE'}
        sentiment = sentiment_mapping.get(video.get('sentiment'), None)
        
        organized_video = {
            'ID': video['video_id'],
            'AUTHOR': video['published_at'],
            'URL': video['url'],
            'DATE': video['published_at'],
            'LIKES': video['like_count'],
            'SHARES': None,  # Nenhuma informação disponível na API sobre compartilhamentos
            'COMMENTS': video['comment_count'],
            'CONTENT': video['description'],
            'SERVICE': 'Youtube',
            'SENTIMENT': sentiment,
            'ENGAGEMENT': None  # Nenhuma informação disponível na API sobre engajamento
        }
        organized_data.append(organized_video)
    return organized_data

def convert_sentiment(sentiment):
    if sentiment == 'NEUTRAL':
        return 0
    elif sentiment == 'POSITIVE':
        return 1
    elif sentiment == 'NEGATIVE':
        return 2
    else:
        return None

In [ ]:
# Recebe os parametros do formulario e inicia a busca
def run_code():
    api_key = api_key_entry.get()
    query = query_entry.get()
    published_after = published_after_entry.get() + "T00:00:00Z"  # Adicionando a hora de início
    published_before = published_before_entry.get() + "T23:59:59Z"  # Adicionando a hora de término
    
    if not (api_key and query and published_after and published_before):
        messagebox.showerror("Erro", "Todos os campos devem ser preenchidos.")
        return
    
    # Buscar vídeos
    videos = search_youtube_videos(api_key, query, published_after, published_before)
    
    if not videos:
        messagebox.showinfo("Informação", "Nenhum vídeo encontrado.")
        return
    
    # Organizar os dados
    organized_videos = organize_video_data(videos)
    
    # Criar o diretório para os arquivos CSV, se não existir
    output_directory = r'C:\madonna\split'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)


In [ ]:
# Converte os dados em arquivos .csv com label pre-definido e salva no diretorio
    # Converter os dados organizados em arquivos CSV individuais
    for index, video in enumerate(organized_videos):
        # Obter os dados do vídeo
        video_id = video['ID']
        service = video['SERVICE']
        date = pd.to_datetime(video['DATE']).strftime('%Y%m%d%H%M')
        sentiment = convert_sentiment(video['SENTIMENT'])
        index_formatted = str(index).zfill(7)  # Formato do índice: 0000000
    
        # Construir o nome do arquivo
        filename = f"{service}_{video_id}_{date}_{sentiment}_{index_formatted}.csv"
        filepath = os.path.join(output_directory, filename)
    
        # Criar DataFrame com apenas uma linha
        df_single_row = pd.DataFrame([video])
    
        # Salvar o DataFrame como arquivo CSV
        df_single_row.to_csv(filepath, index=False)

In [ ]:
# Formulario que recebe as variáveis de busca e inicia o processo
root = tk.Tk()
root.title("Formulário de Entrada")

frame = ttk.Frame(root, padding="20")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

api_key_label = ttk.Label(frame, text="API Key:")
api_key_label.grid(row=0, column=0, sticky=tk.W)
api_key_entry = ttk.Entry(frame, width=30)
api_key_entry.grid(row=0, column=1, sticky=tk.W)

query_label = ttk.Label(frame, text="Query:")
query_label.grid(row=1, column=0, sticky=tk.W)
query_entry = ttk.Entry(frame, width=30)
query_entry.grid(row=1, column=1, sticky=tk.W)

published_after_label = ttk.Label(frame, text="Published After (YYYY-MM-DD):")
published_after_label.grid(row=2, column=0, sticky=tk.W)
published_after_entry = ttk.Entry(frame, width=30)
published_after_entry.grid(row=2, column=1, sticky=tk.W)

published_before_label = ttk.Label(frame, text="Published Before (YYYY-MM-DD):")
published_before_label.grid(row=3, column=0, sticky=tk.W)
published_before_entry = ttk.Entry(frame, width=30)
published_before_entry.grid(row=3, column=1, sticky=tk.W)

submit_button = ttk.Button(frame, text="Buscar Vídeos", command=run_code)
submit_button.grid(row=4, columnspan=2)

root.mainloop()

# Update de codigo para ajustar entendimento da ultima reunião de 20240613. #